In [48]:
import iotbx.pdb
from scitbx.array_family import flex
from cctbx import crystal
import cctbx
import math

#def run(cutoff = 3.5):
cutoff = 1.3
pdb_inp = iotbx.pdb.input(file_name = "1yjp.pdb")
crystal_symmetry = pdb_inp.crystal_symmetry()
hierarchy = pdb_inp.construct_hierarchy()
sites_cart = hierarchy.atoms().extract_xyz()
sst = crystal_symmetry.special_position_settings().site_symmetry_table(
sites_cart = sites_cart)
conn_asu_mappings = crystal_symmetry.special_position_settings().\
asu_mappings(buffer_thickness=5)
conn_asu_mappings.process_sites_cart(
original_sites      = sites_cart,
site_symmetry_table = sst)
# conn_pair_asu_table = cctbx.crystal.pair_asu_table(
# asu_mappings=conn_asu_mappings)
#conn_pair_asu_table.add_all_pairs(distance_cutoff=cutoff)
pair_generator = cctbx.crystal.neighbors_fast_pair_generator(
conn_asu_mappings,
distance_cutoff=cutoff)
fm = crystal_symmetry.unit_cell().fractionalization_matrix()
om = crystal_symmetry.unit_cell().orthogonalization_matrix()

def dist_expl(r1, r2, op):
    r1_f = fm*flex.vec3_double([r1]) # convert to fractional coordinates
    r1_f_mapped = op*r1_f[0] # apply symmetry operator
    r1_c_mapped = (om*flex.vec3_double([r1_f_mapped]))[0] # back to Cartesian
    return math.sqrt( (r1_c_mapped[0]-r2[0])**2 +
                      (r1_c_mapped[1]-r2[1])**2 +
                      (r1_c_mapped[2]-r2[2])**2 )

pairs = []
for pair in pair_generator:
    rt_mx_i = conn_asu_mappings.get_rt_mx_i(pair)
    rt_mx_j = conn_asu_mappings.get_rt_mx_j(pair)
    rt_mx_ji = rt_mx_i.inverse().multiply(rt_mx_j)
    i,j = pair.i_seq, pair.j_seq
    # Compute distance explicitly - just for illustration
    de = dist_expl(r1=sites_cart[j], r2=sites_cart[i], op=rt_mx_ji)
    #
    if rt_mx_ji.is_unit_mx():
        pairs.append((pair.i_seq, pair.j_seq, rt_mx_ji, math.sqrt(pair.dist_sq), de))
    #print(pair.i_seq, pair.j_seq, rt_mx_ji, math.sqrt(pair.dist_sq), de)
print(len(pairs))

14


In [49]:
from sklearn.neighbors import KDTree
tree = KDTree(sites_cart)
inds = tree.query_radius(sites_cart,r=cutoff)
pairs = set()
for i,ind in enumerate(inds):
    for j in ind:
        if i !=j:
            pairs.add(frozenset((i,j)))
print(len(pairs))

13
